# FITS visualisation using Jupyter

### Interactive Visualisation using WebRTC and H.264 Encoding

In [1]:
import cv2
import asyncio
import uvloop
import concurrent
import numpy as np
import bqplot.pyplot as plt
import warnings
from scipy import ndimage
from pyastrovis.webrtc import WebRTCStream
from pyastrovis.widget import WebRTCWidget
from ipywidgets import IntSlider, HBox, VBox, Layout
from IPython.display import display
from astropy.wcs import WCS, FITSFixedWarning
from astropy.io import fits
from astropy.nddata import Cutout2D
from cryptography.utils import CryptographyDeprecationWarning

uvloop.install()
warnings.simplefilter('ignore', FITSFixedWarning)
warnings.simplefilter('ignore', CryptographyDeprecationWarning)

In [2]:
fits_file = './image.fits'
hdu = fits.open(fits_file, lazy_load_hdus=True)[0]
image_data = hdu.data.squeeze()
channels = image_data.shape[0]
w = WCS(hdu.header)

server = await WebRTCWidget.create_server(host='0.0.0.0', port=8080)
panel = server.create_panel(500, 500)

In [3]:
cut = None
chan = 0
pos = (6100, 5400)
size = (500, 500)

def modify(data):
    global cut
    cut = Cutout2D(data, pos, size, wcs=w.celestial)
    img = cut.data
    img = 512*((img-img.min())/img.ptp())
    img = img.astype(np.uint8)
    #img = ndimage.gaussian_filter(img, 1)
    #img = cv2.Canny(img, 300, 400)
    #img = cv2.applyColorMap(img, cv2.COLORMAP_HSV)
    return img

def get_channel_data(value):
    return image_data[value, :,:]

async def channel():
    loop = asyncio.get_running_loop()
    while True:
        global chan
        chan = await WebRTCWidget.wait_for_change(slider, 'value')
        data = await loop.run_in_executor(None, get_channel_data, chan)
        img = await loop.run_in_executor(None, modify, data)
        scale = 'gray8'
        #scale = 'rgb24'
        await panel.add_data(img, scale)

async def spectra():
    while True:
        x, y = await WebRTCWidget.wait_for_change(panel, 'position')
        if cut:
            c_x = x + cut.position_original[0]-cut.position_cutout[0]
            c_y = y + cut.position_original[1]-cut.position_cutout[1]
            spec = [image_data[i, c_y, c_x] for i in range(channels)]
            lines.y = spec

asyncio.ensure_future(channel())
asyncio.ensure_future(spectra())

slider = IntSlider(description='Channel:', layout=Layout(width='60%'), 
                   value=0, min=0, step=2, max=channels-1)

x = [i for i in range(channels)]
y = []
fig = plt.figure(figsize=(3,4))
axes_options = {'x': {'label': 'Channel'}, 'y': {'label': 'Jy/beam'}}
lines = plt.plot(x=x, y=y, colors=['red'], axes_options=axes_options)

display(panel, VBox([slider, fig]))

WebRTCClientWidget(height=500, id='f22feee4-7a66-49cd-85de-f835bec5ff9e', position=(0, 0), url='http://localho…

### Volumentric Rendering

In [ ]:
import warnings
import ipyvolume as ipv
import numpy as np
import scipy.ndimage
from astropy.io import fits
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from astropy.wcs import WCS, FITSFixedWarning

warnings.simplefilter('ignore', FITSFixedWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

pos = (6100, 5400)
size = (500, 500)

fits_file = './image.fits'
hdu = fits.open(fits_file, lazy_load_hdus=True)[0]
data = hdu.data.squeeze()
w = WCS(hdu.header)

channels = [Cutout2D(data[i,:,:], pos, size, wcs=w.celestial).data for i in range(1, 10)]
cube = scipy.ndimage.gaussian_filter(np.dstack(channels), 0.01)

In [ ]:
ipv.quickvolshow(cube, lighting=False, level=[0.29, 0.3, 0.33], opacity=[0.1, 0.1, 0.1], level_width=0.1)